In [14]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

# ベースURL
base_url = "https://www.jalan.net/ikisaki/map/tokyo/"
response = requests.get(base_url)

# メタタグからエンコーディングを取得して設定
soup = BeautifulSoup(response.content, "html.parser")
meta_tag = soup.find("meta", {"charset": True})

if meta_tag and meta_tag.get("charset"):
    response.encoding = meta_tag.get("charset")
else:
    response.encoding = response.apparent_encoding

# 再度パース
soup = BeautifulSoup(response.text, "html.parser")

# 'map-right-inner' を持つ div を取得
map_right_inner_div = soup.find("div", class_="map-right-inner")
if map_right_inner_div:
    a_tags = map_right_inner_div.find_all("a", class_="sub")
    print(f"発見した a タグの数: {len(a_tags)}")
else:
    print("対象の div が見つかりませんでした。")
    a_tags = []

# URLを収集
urls = [urljoin(base_url, a['href']) for a in a_tags if 'href' in a.attrs]

# 特定の文字化けを修正する関数
def fix_encoding(text):
    try:
        return text.encode('latin1').decode('utf-8')
    except (UnicodeEncodeError, UnicodeDecodeError):
        return text  # 修正できない場合そのまま返す

# 各URLにアクセスして詳細情報を取得
for i, url in enumerate(urls, 1):
    print(f"{i}: {url}")
    response = requests.get(url)

    # メタタグでエンコーディングを確認して設定
    detail_soup = BeautifulSoup(response.content, "html.parser")
    meta_tag = detail_soup.find("meta", {"charset": True})
    if meta_tag and meta_tag.get("charset"):
        response.encoding = meta_tag.get("charset")
    else:
        response.encoding = response.apparent_encoding

    # 再度HTMLをパース
    detail_soup = BeautifulSoup(response.text, "html.parser")

    # 各ホテル情報を収集
    hotel_divs = detail_soup.find_all("div", class_="p-yadoCassette__body p-searchResultItem__body")
    print(f"ページ内のホテル数: {len(hotel_divs)}")

    for hotel_div in hotel_divs:
        # ホテル名
        h2_tag = hotel_div.find("h2", class_="p-searchResultItem__facilityName")
        raw_hotel_name = h2_tag.get_text(strip=True) if h2_tag else "ホテル名なし"
        hotel_name = fix_encoding(raw_hotel_name)

        # 詳細ページURL
        a_tag = hotel_div.find("a", class_="jlnpc-yadoCassette__link s16_00 fb")
        detail_url = a_tag['data-href'] if a_tag and 'data-href' in a_tag.attrs else "URLなし"

        # アクセス情報
        access_tag = hotel_div.find("dd", class_="p-searchResultItem__accessValue")
        access_info = access_tag.get_text(strip=True) if access_tag else "アクセス情報なし"

        # 価格情報
        price_tag = hotel_div.find("span", class_="p-searchResultItem__lowestPriceValue")
        price_info = price_tag.get_text(strip=True) if price_tag else "価格情報なし"

        # 一人あたり価格
        unit_price_tag = hotel_div.find("span", class_="p-searchResultItem__lowestUnitPrice")
        unit_price_info = unit_price_tag.get_text(strip=True) if unit_price_tag else "一人あたり価格情報なし"

        # ホテルの評価
        rating_tag = hotel_div.find("span", class_="p-searchResultItem__summaryaverage-num")
        rating_info = rating_tag.get_text(strip=True) if rating_tag else "評価なし"

        # レビュー数
        review_count_tag = hotel_div.find("span", class_="ji ji-bubble-outline")
        review_count_info = review_count_tag.get_text(strip=True) if review_count_tag else "レビュー数なし"

        # 情報を表示
        print(f"ホテル名: {hotel_name}")
        print(f"詳細ページURL: {detail_url}")
        print(f"アクセス情報: {access_info}")
        print(f"価格情報: {price_info}")
        print(f"一人あたり価格: {unit_price_info}")
        print(f"ホテルの評価: {rating_info}")
        print(f"レビュー数: {review_count_info}")
        print("-" * 50)


発見した a タグの数: 13
1: https://www.jalan.net/130000/LRG_137100/
ページ内のホテル数: 32
ホテル名: 儕僢僠儌儞僪儂僥儖搶嫗幣
詳細ページURL: javascript:doYadDetailAd('358206','TGS01419110','115_1_1','2');
アクセス情報: JR丒搶嫗儌僲儗乕儖 昹徏挰墂 杒岥傛傝搆曕栺6暘丅愺憪慄丒戝峕屗慄 戝栧墂 A6弌岥傛傝搆曕栺2暘丅
価格情報: 13,950墌乣
一人あたり価格: 1柤 6,975墌乣
ホテルの評価: 4.4
レビュー数: 
--------------------------------------------------
ホテル名: 僌儔儞僪僯僢僐乕搶嫗 戜応
詳細ページURL: javascript:doYadDetailAd('397480','TGS01419110','115_1_2','2');
アクセス情報: 備傝偐傕傔戜応墂捈寢乛傝傫偐偄慄搶嫗僥儗億乕僩墂壓幵搆曕栺10暘乛昳愳墂偐傜偍戜応儗僀儞儃乕僶僗栺20暘
価格情報: 25,400墌乣
一人あたり価格: 1柤 12,700墌乣
ホテルの評価: 4.6
レビュー数: 
--------------------------------------------------
ホテル名: 昳愳僾儕儞僗儂僥儖
詳細ページURL: URLなし
アクセス情報: 怴姴慄丒JR慄丒嫗媫慄 昳愳墂乮崅椫岥乯傛傝搆曕栺2暘丅嫗媫慄 塇揷嬻峘崙撪慄僞乕儈僫儖墂傛傝嵟懍14暘両
価格情報: 18,072墌乣
一人あたり価格: 1柤 9,036墌乣
ホテルの評価: 4.0
レビュー数: 
--------------------------------------------------
ホテル名: 憡揝僌儔儞僪僼儗僢僒 搶嫗儀僀桳柧乮媽儂僥儖僒儞儖乕僩桳柧乯
詳細ページURL: URLなし
アクセス情報: 傝傫偐偄慄崙嵺揥帵応墂丄備傝偐傕傔搶嫗價僢僌僒僀僩墂偐傜搆曕3暘丅庱搒崅榩娸慄桳柧儔儞僾傛傝3暘丅
価格情報: 11,050墌乣
一人あたり価格: 1柤 5,525墌乣
ホテルの評価: 4.1
レビュー数: 
---------------------------------------